In [1]:
%gui qt

In [2]:
import matplotlib as mpl
import numpy as np
from vispy import scene, app

from PyQt5 import QtCore
from PyQt5.QtCore import Qt, QThread, QEventLoop
from PyQt5.QtWidgets import QMainWindow, QAction, QFileDialog, QWidget, QSplitter, QComboBox, QTextBrowser, QSlider, QPushButton, QTableWidget, QVBoxLayout, QHBoxLayout, QLineEdit, QLabel, QGridLayout
from PyQt5.QtGui import QIcon
from PyQt5.QtWidgets import QApplication


class LineView(scene.SceneCanvas):
    def __init__(self):
        super().__init__(keys=None)
        self.unfreeze()
        
        # Initialize grid and view
        self.grid = self.central_widget.add_grid(margin=10)
        self.view = self.grid.add_view(row=0, col=1)
        self.view.camera = 'panzoom'
        
        # Initialize line collections
        self.lines = []
        self.line_colors = mpl.colormaps.get_cmap('Set3').colors
        
        self._setup_axes()
        self.freeze()

    def _setup_axes(self, axis_color=(1, 1, 1, 0.8)):
        """Setup x and y axes with common properties"""
        # Create axes with shared properties 
        axis_props = {
            'text_color': axis_color,
            'axis_color': axis_color,
            'tick_color': axis_color
        }
        
        self.xaxis = scene.AxisWidget(orientation='bottom', **axis_props)
        self.yaxis = scene.AxisWidget(orientation='left', **axis_props)
        
        # Configure axes
        self.xaxis.stretch = (0.9, 0.1)
        self.yaxis.stretch = (0.1, 0.9)
        
        # Add to grid and link views
        self.grid.add_widget(self.xaxis, row=1, col=1)
        self.grid.add_widget(self.yaxis, row=0, col=0)
        self.xaxis.link_view(self.view)
        self.yaxis.link_view(self.view)
    
    def set_data(self, positions):
        """Set line data and update view
        
        Parameters
        ----------
        positions : array-like
            List of position arrays for each line
        """
        self.clear()
        n_colors = len(self.line_colors)
        
        for i, pos in enumerate(positions):
            color = self.line_colors[i % n_colors]
            line = scene.visuals.Line(pos=pos, color=color, width=2)
            self.lines.append(line)
            self.view.add(line)
            
        self.view.camera.set_range()
    
    def clear(self):
        """Remove all lines from view"""
        for line in self.lines:
            line.parent = None
        self.lines.clear()

class GUI(QWidget):
    def __init__(self):
        super().__init__()

        self.plot = LineView()
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update)

        self.btn = QPushButton('Start')
        self.btn.setCheckable(True)
        self.btn.toggled.connect(self.start)

        layout_main = QVBoxLayout()
        layout_main.addWidget(self.btn)
        layout_main.addWidget(self.plot.native)
        self.setLayout(layout_main)
        self.data = None
        self.time = 0

    def set_data(self, pos):
        self.data = pos
    
    def update(self):
        self.plot.set_data([self.data[self.time:self.time+1000]])
        self.time += 1

    def start(self, state):
        if state:
            self.timer.start(200)
        else:
            self.timer.stop()

In [3]:
pos_x = np.arange(100000)
pos_y = np.random.randn(100000)

gui = GUI()
gui.set_data(np.stack([pos_x, pos_y], axis=1))
gui.show()

In [9]:
import matplotlib as mpl
x = mpl.colormaps.get_cmap('Set3').colors
x

((0.5529411764705883, 0.8274509803921568, 0.7803921568627451),
 (1.0, 1.0, 0.7019607843137254),
 (0.7450980392156863, 0.7294117647058823, 0.8549019607843137),
 (0.984313725490196, 0.5019607843137255, 0.4470588235294118),
 (0.5019607843137255, 0.6941176470588235, 0.8274509803921568),
 (0.9921568627450981, 0.7058823529411765, 0.3843137254901961),
 (0.7019607843137254, 0.8705882352941177, 0.4117647058823529),
 (0.9882352941176471, 0.803921568627451, 0.8980392156862745),
 (0.8509803921568627, 0.8509803921568627, 0.8509803921568627),
 (0.7372549019607844, 0.5019607843137255, 0.7411764705882353),
 (0.8, 0.9215686274509803, 0.7725490196078432),
 (1.0, 0.9294117647058824, 0.43529411764705883))

IndexError: tuple index out of range